In [137]:
# Import libraries
import pandas as pd     
import numpy as np
from pytrends.request import TrendReq


In [240]:
# Create lists of keywords
list1 = ['Weather', 'Coronavirus', 'Pizza', 'Stimulus', 'Trump']
list2 = ['Weather', 'George Floyd', 'Protest', 'Biden', 'Police']

In [241]:
# Use pytrends to build payload queries
pytrends1 = TrendReq(hl='en-US', tz=360)
pytrends2 = TrendReq(hl='en-US', tz=360)

pytrends1.build_payload(list1, cat=0, timeframe = '2020-01-01 2020-08-13', geo='US', gprop='')
pytrends2.build_payload(list2, cat=0, timeframe = '2020-01-01 2020-08-13', geo='US', gprop='')

In [242]:
# Get Google interest over time data
df1=pytrends1.interest_over_time()
df1=df1.drop(['isPartial'], axis =1)

df2=pytrends2.interest_over_time()
df2=df2.drop(['isPartial'], axis =1)

In [152]:
# Calculate multiplier to make both datasets proportional
multiplier = df2['Weather'] / df1['Weather']
multiplier = multiplier.mean()
multiplier

2.9510466757413156

In [153]:
# Update df1
df1_updated = (df1 * multiplier).round().astype(np.int64)
df1_updated = df1_updated.drop(['Weather'], axis = 1)
df1_updated

,Coronavirus,Pizza,Stimulus,Trump
date,,,,
2020-01-01,0,27,0,9
2020-01-02,0,15,0,6
2020-01-03,0,18,0,18
2020-01-04,0,21,0,15
2020-01-05,0,21,0,15
...,...,...,...,...
2020-08-07,18,21,24,15
2020-08-08,15,24,24,21
2020-08-09,18,21,18,21


In [154]:
df2

,Weather,George Floyd,Protest,Biden,Police
date,,,,,
2020-01-01,49,0,0,1,5
2020-01-02,53,0,0,1,5
2020-01-03,51,0,0,1,5
2020-01-04,53,0,0,0,4
2020-01-05,50,0,0,0,4
...,...,...,...,...,...
2020-08-07,45,1,0,6,5
2020-08-08,45,1,1,6,5
2020-08-09,44,1,1,6,5


In [201]:
# Merge 2 datasets together on date
df_merge = pd.merge(df1_updated, df2, on='date')
df_merge

,Coronavirus,Pizza,Stimulus,Trump,Weather,George Floyd,Protest,Biden,Police
date,,,,,,,,,
2020-01-01,0,27,0,9,49,0,0,1,5
2020-01-02,0,15,0,6,53,0,0,1,5
2020-01-03,0,18,0,18,51,0,0,1,5
2020-01-04,0,21,0,15,53,0,0,0,4
2020-01-05,0,21,0,15,50,0,0,0,4
...,...,...,...,...,...,...,...,...,...
2020-08-07,18,21,24,15,45,1,0,6,5
2020-08-08,15,24,24,21,45,1,1,6,5
2020-08-09,18,21,18,21,44,1,1,6,5


In [202]:
# Reset index
df_merge=df_merge.reset_index()

In [203]:
# Melt data to bring column headers into rows
df_merge=df_merge.melt('date', value_name='value', var_name = 'name')

In [204]:
# Sort values by date then name
df_merge=df_merge.sort_values(['date', 'name'])
df_merge.head(20)

,date,name,value
1568,2020-01-01,Biden,1
0,2020-01-01,Coronavirus,0
1120,2020-01-01,George Floyd,0
224,2020-01-01,Pizza,27
1792,2020-01-01,Police,5
1344,2020-01-01,Protest,0
448,2020-01-01,Stimulus,0
672,2020-01-01,Trump,9
896,2020-01-01,Weather,49
1569,2020-01-02,Biden,1


In [221]:
len(df_merge)

2016

In [222]:
# Create series of values to manipulate
last_value = df_merge['value'].to_list()
len(last_value)

2016

In [223]:
# Delete last 9 values from series
del last_value[(len(df_merge)-10):-1]
len(last_value)

2007

In [224]:
# Insert 9 '0' to top of list to create lastValue 
for i in range(9):
   last_value.insert(0, 0)

len(last_value)

2016

In [230]:
# Insert lastValue into new column in df
df_merge['lastValue'] = last_value
df_merge.reset_index(drop=True, inplace=True)
df_merge.head(20)


,date,name,value,lastValue
0,2020-01-01,Biden,1,0
1,2020-01-01,Coronavirus,0,0
2,2020-01-01,George Floyd,0,0
3,2020-01-01,Pizza,27,0
4,2020-01-01,Police,5,0
5,2020-01-01,Protest,0,0
6,2020-01-01,Stimulus,0,0
7,2020-01-01,Trump,9,0
8,2020-01-01,Weather,49,0
9,2020-01-02,Biden,1,1


In [231]:
# Export to csv
df_merge.to_csv(path_or_buf = 'pytrends.csv')

In [245]:
# df2.reset_index().plot(x='date', y='Police', figsize=(120, 10), kind ='bar')